# Preparación del Dataset
---

- Se obtienen y preprocesan textos directos del dataset para luego ser persistidos.

In [21]:
import pandas as pd
import os
from document import Document
import util.log as log
from multiprocessing import Process
from multiprocessing import Manager
from multiprocessing import Pool
import pandas as pd
from datetime import datetime
log.init_logger()

In [22]:
corpora = []
init_time = datetime.now()
directory = 'labeled_dataset/'
files = os.listdir(directory)
dataframe = None
number_of_cores_to_use = 10

def process_file(file):
    log.info('Processing file: {}'.format(file))
    try:
        path = '{}{}'.format(directory, file)
        document = Document(path)
        splitted_file = file.split(' $ ') 
        topic = splitted_file[0]
        title = splitted_file[1]
        return [path, title, document.string, document.word_count(), document.type_count(), document.sentences,\
                       document.lemmatized_string, document.stemmed_string, document.simple_preprocessed_string, topic, \
                       document.named_entities, document.bigrams, document.trigrams, document.lemmatized_bigrams, \
                       document.lemmatized_trigrams, document.stemmed_bigrams, document.stemmed_trigrams, \
                       document.simple_preprocessed_bigrams, document.simple_preprocessed_trigrams]
    except InvalidDocument:
        pass



with Pool(number_of_cores_to_use) as pool:
    corpora = pool.map(process_file, files)

time = datetime.now() - init_time
log.info('{} documents were processed. {} documents errored. Total time used: {}, Total cores used: {}'.format(len(corpora), len(errors), \
                                                                                                          str(time), number_of_cores_to_use))

In [23]:
dataframe = pd.DataFrame(data = corpora, columns=['original_path', 'document_title', 'string', 'word_count', 'type_count', 'sentences','lemmatized_text', 'stemmed_text', 'simple_preprocessed', 'topic', 'named_entities', 'tokens_bigrams', 'tokens_trigrams', 'lemmatized_bigrams', 'lemmatized_trigrams', 'stemmed_bigrams', 'stemmed_trigrams', 'simple_preprocessed_bigrams', 'simple_preprocessed_trigrams'])

In [24]:
dataframe.to_csv('data.csv')